<a href="https://colab.research.google.com/github/AdamJelley/AdventOfCode2021/blob/main/Day19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 19

In [95]:
import requests
import numpy as np
import itertools

In [122]:
#Variables to set
day = 19
cookie = ''

In [3]:
input_link = f'https://adventofcode.com/2021/day/{day}/input'
user_cookie = {'session':cookie} # Retrieve session cookie corresponding to user login (by inspecting cookies on data page)

In [123]:
test_input = """{COPY FROM CHALLENGE}"""

In [60]:
raw_input = requests.get(input_link, cookies=user_cookie).text[:-1]

In [56]:
def process_input(input):
  processed_input = [[np.array(coordinates.split(','), dtype=np.int) for coordinates in scanner.split('---')[2][1:].split('\n')] for scanner in input.split('\n\n')]
  return processed_input

In [61]:
test_beacons = process_input(test_input)
real_beacons = process_input(raw_input)

## Part 1

In [12]:
def check_scanner_overlap(scanner1, scanner2):
  overlap=False
  same_beacons = []
  for i in range(len(scanner1)):
    for j in range(len(scanner2)):
      overlap_counter=0
      scanner1_relative = [beacon-scanner1[i] for beacon in scanner1]
      scanner2_relative = [beacon-scanner2[j] for beacon in scanner2]
      for beacon in scanner1_relative:
        if np.any(np.all(np.sort(np.abs(beacon))==np.sort(np.abs(scanner2_relative)), axis=1)):
          overlap_counter+=1 # Relative absolute distance between two beacons is the same for the two scanners
      if overlap_counter>11: # If there are 12 or more beacons that are the same relative distance away, then beacons must be the same and scanners overlap
        overlap=True
        same_beacons.append((i,j))
  return overlap, same_beacons

In [72]:
def find_overlapping_scanners(input_data):
  overlap_dict = {}
  same_beacons_dict = {}
  for i, scanner1 in enumerate(input_data):
    for j, scanner2 in enumerate(input_data):
      if i!=j:
        overlap, same_beacons = check_scanner_overlap(scanner1, scanner2)
        overlap_dict[(i,j)] = overlap
        if overlap:
          same_beacons_dict[(i,j)] = same_beacons
  return overlap_dict, same_beacons_dict

In [21]:
def rotations():
  for x, y, z in itertools.permutations([0, 1, 2]):
    for sx, sy, sz in itertools.product([-1, 1], repeat=3):
      rotation_matrix = np.zeros((3, 3))
      rotation_matrix[0, x] = sx
      rotation_matrix[1, y] = sy
      rotation_matrix[2, z] = sz
      if np.linalg.det(rotation_matrix) == 1:
        yield rotation_matrix

In [94]:
def calculate_scanner_relative_location(scanner1, scanner2, same_beacons):
  scanner2_location = []
  scanner2_orientation = []
  for beacon in same_beacons:
    scanner1_measurement = scanner1[beacon[0]] # Assume scanner 1 is at (0,0,0) with upright orientation
    scanner2_measurement = scanner2[beacon[1]]
    possible_scanner2_locations = []
    possible_scanner2_orientations = []
    
    for rotation in rotations():
      scanner2_permutation = np.matmul(rotation, scanner2_measurement)
      possible_scanner2_locations.append(scanner1_measurement-scanner2_permutation) # Get scanner 2 location for all possible orientiations
      possible_scanner2_orientations.append(rotation)      

      if scanner2_location == []:
        scanner2_location = possible_scanner2_locations # Initialise scanner 2 location to all possible locations for first beacon
        scanner2_orientation = possible_scanner2_orientations
    for i, location in enumerate(scanner2_location):
      if not np.any(np.all(location==possible_scanner2_locations, axis=1)):
        scanner2_location.pop(i) # For each consistent beacon, only keep possible consistent scanner locations
        scanner2_orientation.pop(i)
  assert len(scanner2_location)==1 # Should only be one possible position left after considering 12 consistent beacons
  return scanner2_location[0], scanner2_orientation[0]

In [28]:
def calculate_scanner_absolute_location(scanner1_location, scanner1_orientation, scanner2_relative_location, scanner2_relative_orientation):
  scanner2_location = scanner1_location + np.matmul(scanner1_orientation, scanner2_relative_location)
  scanner2_rotation = np.matmul(scanner1_orientation, scanner2_relative_orientation)
  return scanner2_location, scanner2_rotation

In [76]:
def get_scanner_locations(input_data, overlap_dict, same_beacons_dict):
  scanner_locations = {0:np.array([0,0,0])}
  scanner_orientations = {0:np.array([[1.,  0., 0.], [0.,  1., 0.], [0.,  0., 1.]])}

  while sorted(list(scanner_locations.keys())) != list(range(len(input_data))):
    for scanner_pair, overlap in overlap_dict.items():
      if overlap:
        (i,j) = scanner_pair
        if i in scanner_locations.keys() and j not in scanner_locations.keys():   
          same_beacons = same_beacons_dict[scanner_pair]
          scanner2_relative_location, scanner2_relative_orientation = calculate_scanner_relative_location(input_data[i], input_data[j], same_beacons)
          scanner_locations[j], scanner_orientations[j] = calculate_scanner_absolute_location(scanner_locations[i], scanner_orientations[i], scanner2_relative_location, scanner2_relative_orientation)
  return scanner_locations, scanner_orientations

In [41]:
def calculate_absolute_beacon_coordinates(scanner_location, scanner_orientation, scanner_coordinates):
  absolute_coordinates = scanner_location + np.matmul(scanner_coordinates, np.linalg.inv(scanner_orientation))
  return absolute_coordinates

In [47]:
def get_unique_beacon_coordinates(data_input, scanner_locations, scanner_orientations):
  all_beacon_coordinates = []
  for i in range(len(data_input)):
    absolute_beacon_coordinates = calculate_absolute_beacon_coordinates(scanner_locations[i], scanner_orientations[i], data_input[i])
    all_beacon_coordinates.append(absolute_beacon_coordinates)
  all_beacon_coordinates = np.concatenate(all_beacon_coordinates)
  unique_beacons = np.unique(all_beacon_coordinates, axis=0)
  return unique_beacons

In [96]:
def solve_input(data_input):
  overlap_dict, same_beacons_dict = find_overlapping_scanners(data_input)
  scanner_locations, scanner_orientations = get_scanner_locations(data_input, overlap_dict, same_beacons_dict)
  unique_beacons = get_unique_beacon_coordinates(data_input, scanner_locations, scanner_orientations)
  return scanner_locations, scanner_orientations, unique_beacons

In [103]:
test_scanner_locations, test_scanner_orientations, test_unique_beacons = solve_input(test_beacons)

In [105]:
print(f'Number of unique beacons for test data: {test_unique_beacons.shape[0]}')

Number of unique beacons for test data: 79


In [101]:
scanner_locations, scanner_orientations, unique_beacons = solve_input(real_beacons)

In [111]:
print(f'Number of unique beacons for input data: {unique_beacons.shape[0]}')

Number of unique beacons for input data: 303


## Part 2

In [121]:
scanner_separations = {}
for scanner1, scanner_location1 in scanner_locations.items():
  for scanner2, scanner_location2 in scanner_locations.items():
    if scanner1>scanner2:
      scanner_separations[(scanner1, scanner2)] = sum(np.abs(scanner_location1-scanner_location2))
scanners_max_separation = max(scanner_separations, key=lambda x:scanner_separations[x])
max_separation = scanner_separations[scanners_max_separation]
print(f'Scanners with largest Manhatten distance {scanners_max_separation} with separation {int(max_separation)}')

Scanners with largest Manhatten distance (22, 5) with separation 9621
